In [1]:
import os
import keras
from test_model_landcover import run_model_on_tile
import time
import rasterio
import numpy as np

Using TensorFlow backend.


In [2]:
def run_model_on_tile(model, naip_tile, inpt_size, output_size, batch_size):
    down_weight_padding = 40
    height = naip_tile.shape[0]
    width = naip_tile.shape[1]

    stride_x = inpt_size - down_weight_padding*2
    stride_y = inpt_size - down_weight_padding*2

    output = np.zeros((height, width, output_size), dtype=np.float32)
    counts = np.zeros((height, width), dtype=np.float32) + 0.000000001
    kernel = np.ones((inpt_size, inpt_size), dtype=np.float32) * 0.1
    kernel[10:-10, 10:-10] = 1
    kernel[down_weight_padding:down_weight_padding+stride_y,
           down_weight_padding:down_weight_padding+stride_x] = 5

    batch = []
    batch_indices = []
    batch_count = 0

    for y_index in (list(range(0, height - inpt_size, stride_y)) + [height - inpt_size,]):
        for x_index in (list(range(0, width - inpt_size, stride_x)) + [width - inpt_size,]):
            naip_im = naip_tile[y_index:y_index+inpt_size, x_index:x_index+inpt_size, :]

            batch.append(naip_im)
            batch_indices.append((y_index, x_index))
            batch_count+=1

    model_output = model.predict(np.array(batch), batch_size=batch_size, verbose=0)

    for i, (y, x) in enumerate(batch_indices):
        output[y:y+inpt_size, x:x+inpt_size] += model_output[i] * kernel[..., np.newaxis]
        counts[y:y+inpt_size, x:x+inpt_size] += kernel

    return output / counts[..., np.newaxis]

In [3]:
input_fn = "c525b58f-89ea-4d80-9681-feabe4cf75e2_ms.tif"
data_dir = os.path.dirname(input_fn)
model_fn = "results/results_epochs_20_5/train-output_ny_1m_2013/final_model.h5"
save_probabilities = True
    
model = keras.models.load_model(model_fn, custom_objects={
        "jaccard_loss":keras.metrics.mean_squared_error,
        "loss":keras.metrics.mean_squared_error
    })
    
output_shape = model.output_shape[1:]
input_shape = model.input_shape[1:]
model_input_size = input_shape[0]

In [4]:
tic = float(time.time())
naip_fn = os.path.join(data_dir, input_fn)


print("Running model on %s" % naip_fn)

naip_fid = rasterio.open(naip_fn, "r")
naip_profile = naip_fid.meta.copy()
print(naip_fid.read().shape)
naip_tile = np.clip(naip_fid.read() / 5000, 0, 5000)

Running model on c525b58f-89ea-4d80-9681-feabe4cf75e2_ms.tif
(4, 505, 1603)


In [5]:
#naip_tile = naip_tile.astype(np.float32) / 255.0
naip_tile = np.rollaxis(naip_tile, 0, 3)
naip_fid.close()

output = run_model_on_tile(model, naip_tile, model_input_size, output_shape[2], 16)
#output[:,:,4] += output[:,:,5]
#output[:,:,4] += output[:,:,6]
output = output[:,:,:5]

#----------------------------------------------------------------
# Write out each softmax prediction to a separate file
#----------------------------------------------------------------
if save_probabilities:
    output_fn = os.path.basename(naip_fn)[:-4] + "_prob.tif"
    current_profile = naip_profile.copy()
    current_profile['driver'] = 'GTiff'
    current_profile['dtype'] = 'uint8'
    current_profile['count'] = 5
    current_profile['compress'] = "lzw"

    # quantize the probabilities
    bins = np.arange(256)
    bins = bins / 255.0
    output = np.digitize(output, bins=bins, right=True).astype(np.uint8)

    f = rasterio.open(os.path.join(output_fn), 'w', **current_profile)
    f.write(output[:,:,0], 1)
    f.write(output[:,:,1], 2)
    f.write(output[:,:,2], 3)
    f.write(output[:,:,3], 4)
    f.write(output[:,:,4], 5)
    f.close()

#----------------------------------------------------------------
# Write out the class predictions
#----------------------------------------------------------------
output_classes = np.argmax(output, axis=2).astype(np.uint8)
output_class_fn = os.path.basename(naip_fn) + "_class.tif"

current_profile = naip_profile.copy()
current_profile['driver'] = 'GTiff'
current_profile['dtype'] = 'uint8'
current_profile['count'] = 1
current_profile['compress'] = "lzw"
f = rasterio.open(os.path.join(output_class_fn), 'w', **current_profile)
f.write(output_classes, 1)
f.close()

print("Finished iteration in %0.4f seconds" % (time.time() - tic))

Finished iteration in 8.2530 seconds
